# Metrics Computation

This notebook computes and analyzes trajectory generation **privacy and utility** metrics for the APU-TrajGen2 model using the Porto or San Francisco datasets. 

##### **Imports**

In [1]:
import os
# oneDNN warning suppression TF 2.4.1
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import copy

import numpy as np
import pandas as pd
import pickle
import statistics

from scipy.stats import energy_distance, wasserstein_distance
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from math import radians
from sklearn.metrics.pairwise import haversine_distances
from fastdtw import fastdtw

from utils.data import *
from utils.plots import *
from utils.metrics import *
from models import *
from apu_trajgen import *

import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

##### **Parameters**

In [2]:
selected_dataset = "PORTO"
# selected_dataset = "SANFRANCISCO"

# Set the number of trajectories to process
n_trajs = 10 # 5000

mean_min = [80, 170, 250, 350] # privacy contraint
mean_max = [155, 350, 450, 550] # utility contraint

# grid_num = 6
grid_num = 20

##### **Metrics computations (fixed k)**

In [3]:
print("Selected dataset: " + selected_dataset)
print("grid_num: " + str(grid_num))

# Load the data and plot trajectory with id=traj_idx
Y_test = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_Y_test_fixed_k_ntrajs_" + str(n_trajs) + ".pkl")
test_seq_len = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_test_seq_len_fixed_k_ntrajs_" + str(n_trajs) + ".pkl")

normalization_ranges = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_normalization_ranges_test.pkl") # the scaler used to normalize the data
normalization_ranges = {"min": normalization_ranges["min"][0:2], "max": normalization_ranges["max"][0:2]}

# Denormalize the data using the scaler or normalization ranges
Y_test_dn = denormalize_data(dataset = copy.deepcopy(Y_test), normalization_ranges = normalization_ranges)
Y_test_dn_list = array_to_list(Y_test_dn)

# Transform the data to grid
min_x, min_y, max_x, max_y = compute_min_max_coordinates(Y_test_dn)
grid_map = grid.GridMap(grid_num, min_x, min_y, max_x, max_y)
Y_test_dn_grid = convert_raw_to_grid(Y_test_dn_list, grid_map, interp=True)
Y_test_dn_sampled = convert_grid_to_raw(Y_test_dn_grid) 

for k in range(1, 6):
    
    Y_pred_k = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_Y_pred_fixed_k_" + str(k) +"_ntrajs_" + str(n_trajs) + ".pkl")
    
    # Denormalize the data using the scaler or normalization ranges
    Y_pred_k_dn = denormalize_data(dataset = Y_pred_k, normalization_ranges = normalization_ranges)
    
    Y_pred_dn_list = array_to_list(Y_pred_k_dn)
    Y_pred_dn_grid = convert_raw_to_grid(Y_pred_dn_list, grid_map,interp=True)
    
    test_density = get_real_density(Y_test_dn_grid, grid_map)
    pred_density = get_real_density(Y_pred_dn_grid, grid_map)
    test_density /= np.sum(test_density)
    pred_density /= np.sum(pred_density)
    density_error = compute_js_distance_prob(test_density, pred_density)

    # Compute Relative Query Error')
    queries = [SquareQuery(grid_map.min_x, grid_map.min_y, grid_map.max_x, grid_map.max_y, size_factor=9) for _ in range(200)]
    query_error = experiment.calculate_point_query(Y_test_dn_sampled,
                                                Y_pred_dn_list,
                                                queries)

    # Compute Hotspot Query Error
    hotspot_ndcg = experiment.calculate_hotspot_ndcg(test_density, pred_density)
    hotspot_qerr = 1 - hotspot_ndcg

    # # Compute Kendall-tau
    kendall_tau = calculate_coverage_kendall_tau(Y_test_dn_grid,
                                                            Y_pred_dn_grid,
                                                            grid_map)

    # Compute Trip error
    test_trip_dist, _, _ = get_start_end_dist(Y_test_dn_grid, grid_map)
    pred_trip_dist, _, _ = get_start_end_dist(Y_pred_dn_grid, grid_map)

    test_trip_dist = np.asarray(test_trip_dist) / np.sum(test_trip_dist)
    pred_trip_dist = np.asarray(pred_trip_dist) / np.sum(pred_trip_dist)
    trip_error = compute_js_distance_prob(test_trip_dist, pred_trip_dist)

    # Compute Length error
    length_error = experiment.calculate_length_error(Y_test_dn_list, Y_pred_dn_list)

    # Compute Diameter error
    diameter_error = experiment.calculate_diameter_error(Y_test_dn_list, Y_pred_dn_list, bucket_num=grid_num,
                                                        multi=False)

    # Pattern mining errors
    test_pattern = experiment.mine_patterns(Y_test_dn_grid)
    pred_pattern = experiment.mine_patterns(Y_pred_dn_grid)

    pattern_f1_error = experiment.calculate_pattern_f1_error(test_pattern, pred_pattern)
    pattern_support_error = experiment.calculate_pattern_support(test_pattern, pred_pattern)
    #

    haversine_dists = []
    haus_dists = []
    edr_dists = []
    dtw_dists = []
    kendalltau_coeffs = []
    js_dists = []
    mi_scores = []
    kl_divs = []


    for traj_idx in range(len(Y_test_dn)):
            
        # Trajectory-level metrics
        # Haversine point-to-point distance
        haversine = compute_point_to_point_haversine_distances(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=float), Y_pred_k_dn[traj_idx])
        
        # Jensen-Shannon distance
        js_dist = compute_js_distance(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=float), Y_pred_k_dn[traj_idx])
        # KL divergence
        kl_div = compute_kl_divergence(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=float), Y_pred_k_dn[traj_idx])
        
        # Mutual Information Score
        mi_score = mutual_information_gps(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=float), Y_pred_k_dn[traj_idx],
                                          n_lat_bins=grid_num, n_lon_bins=grid_num)
        #

        haversine_dists.append(np.mean(haversine))
        
        js_dists.append(js_dist)
        kl_divs.append(kl_div)
        mi_scores.append(mi_score)
        
        # Hausdorff distance
        hdist = dst_utils_tsgan.hausdorff_distance(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=float) , Y_pred_k_dn[traj_idx], distance="haversine")

        # EDR distance
        edr = dst_utils_tsgan.edit_distance(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], Y_pred_k_dn[traj_idx], eps=0.01)
        # DTW distance
        dtw, path = fastdtw(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], Y_pred_k_dn[traj_idx], dist=haversine_dist_in_km)
        
        haus_dists.append(hdist)
        edr_dists.append(edr)
        dtw_dists.append(dtw)
        

    print("Method: APU-TrajGen2 - fixed k = "+ str(k))
    print("---")
    print("Mean Density Error:")
    print(density_error)
    print("Mean Query Error:")
    print(query_error)
    print("Mean Hotspot Query Error:")
    print(hotspot_qerr)
    print("Mean Coverage Kendall Tau:")
    print(kendall_tau)
    print("---")
    print("Mean Trip Error:")
    print(trip_error)
    print("Mean Length Error:")
    print(length_error) 
    print("Mean Diameter Error:")
    print(diameter_error) 
    print("Mean Hausdorff:")
    print(np.mean(haus_dists))
    print("Mean DTW:")
    print(np.mean(dtw_dists)) 
    print("Mean EDR:")
    print(np.mean(edr_dists))
    print("Mean Haversine distance (MDE):")
    print(np.mean(haversine_dists)/1000)  # Convert to km
    print("---")
    print("Mean Pattern F1 Error:")
    print(pattern_f1_error)
    print("Mean Pattern Support Error:")
    print(pattern_support_error)
    print("-------")
    print("Mutual Information Score:")
    print(np.mean(mi_scores))
    
    print("##################################")
    
    

Selected dataset: PORTO
grid_num: 20
Method: APU-TrajGen2 - fixed k = 1
---
Mean Density Error:
0.07926993386524774
Mean Query Error:
0.8886006023159305
Mean Hotspot Query Error:
0.5442095754346259
Mean Coverage Kendall Tau:
0.3358521303258145
---
Mean Trip Error:
0.4158882483359762
Mean Length Error:
0.3327948025332016
Mean Diameter Error:
0.3138705462946926
Mean Hausdorff:
0.2197772261977661
Mean DTW:
3.146948045879319
Mean EDR:
0.0
Mean Haversine distance (MDE):
0.09466352382318971
---
Mean Pattern F1 Error:
0.45
Mean Pattern Support Error:
0.45
-------
Mutual Information Score:
2.9296613943464274
##################################
Method: APU-TrajGen2 - fixed k = 2
---
Mean Density Error:
0.1468562801510516
Mean Query Error:
0.97143036384682
Mean Hotspot Query Error:
1.0
Mean Coverage Kendall Tau:
0.25144110275689224
---
Mean Trip Error:
0.5545176644479682
Mean Length Error:
0.6931471305599506
Mean Diameter Error:
0.26009405639226046
Mean Hausdorff:
0.723837780834623
Mean DTW:
10.4

##### **Metrics computations (adaptive k)**

In [4]:
print("Selected dataset: " + selected_dataset)
print("grid_num: " + str(grid_num))

# Load the data 
Y_test = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_Y_test_adaptive_k_ntrajs_" + str(n_trajs) + ".pkl")
test_seq_len = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_test_seq_len_adaptive_k_ntrajs_" + str(n_trajs) + ".pkl")

normalization_ranges = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_normalization_ranges_test.pkl") # the scaler used to normalize the data
normalization_ranges = {"min": normalization_ranges["min"][0:2], "max": normalization_ranges["max"][0:2]}

# Denormalize the data using the scaler or normalization ranges
Y_test_dn = denormalize_data(dataset = copy.deepcopy(Y_test), normalization_ranges = normalization_ranges)

# Transform the data to grid
min_x, min_y, max_x, max_y = compute_min_max_coordinates(Y_test_dn)
grid_map = grid.GridMap(grid_num, min_x, min_y, max_x, max_y)
Y_test_dn_list = array_to_list(Y_test_dn)
Y_test_dn_grid = convert_raw_to_grid(Y_test_dn_list, grid_map, interp=False)

for mmin, mmax in zip(mean_min, mean_max):
    
    Y_pred_k = load_pickle(DATA_FOLDER + selected_dataset.lower() + "_Y_pred_adaptive_k_mean_min" + str(mmin) 
             + "_mean_max" + str(mmax) + "_ntrajs_" + str(n_trajs) + ".pkl")
    
    # Denormalize the data using the scaler or normalization ranges
    Y_pred_k_dn = denormalize_data(dataset = Y_pred_k, normalization_ranges = normalization_ranges)
    
    # Global level metrics
    # Compute density error
    Y_pred_dn_list = array_to_list(Y_pred_k_dn)
    Y_pred_dn_grid = convert_raw_to_grid(Y_pred_dn_list, grid_map,interp=False)
    test_density = get_real_density(Y_test_dn_grid, grid_map)
    pred_density = get_real_density(Y_pred_dn_grid, grid_map)
    test_density /= np.sum(test_density)
    pred_density /= np.sum(pred_density)
    density_error = compute_js_distance_prob(test_density, pred_density)
    
    # Compute Relative Query Error')

    queries = [SquareQuery(grid_map.min_x, grid_map.min_y, grid_map.max_x, grid_map.max_y, size_factor=9) for _ in range(200)]
    query_error = experiment.calculate_point_query(Y_test_dn_list,
                                                Y_pred_dn_list,
                                                queries)
    
    # Compute Hotspot Query Error
    hotspot_ndcg = experiment.calculate_hotspot_ndcg(test_density, pred_density)
    hotspot_qerr = 1 - hotspot_ndcg

    # Compute Trip error
    test_trip_dist, _, _ = get_start_end_dist(Y_test_dn_grid, grid_map)
    pred_trip_dist, _, _ = get_start_end_dist(Y_pred_dn_grid, grid_map)

    test_trip_dist = np.asarray(test_trip_dist) / np.sum(test_trip_dist)
    pred_trip_dist = np.asarray(test_trip_dist) / np.sum(pred_trip_dist)
    trip_error = compute_js_distance_prob(test_trip_dist, pred_trip_dist)
    
    # Compute Length error
    length_error = experiment.calculate_length_error(Y_test_dn_list, Y_pred_dn_list)
    
    # Compute Diameter error
    diameter_error = experiment.calculate_diameter_error(Y_test_dn_list, Y_pred_dn_list, bucket_num=grid_num,
                                                        multi=False)

    # Pattern mining errors
    test_pattern = experiment.mine_patterns(Y_test_dn_grid)
    pred_pattern = experiment.mine_patterns(Y_pred_dn_grid)

    pattern_f1_error = experiment.calculate_pattern_f1_error(test_pattern, pred_pattern)
    pattern_support_error = experiment.calculate_pattern_support(test_pattern, pred_pattern)
    #
    
    haversine_dists = []
    haus_dists = []
    edr_dists = []
    dtw_dists = []
    kendalltau_coeffs = []
    js_dists = []
    mi_scores = []
    kl_divs = []
 
    n_mu_n = 0
    n_traj = 0
    
    for traj_idx in range(n_trajs):
        
        # Trajectory-level metrics
        # print("Trajectory: " + str(traj_idx))
        # Haversine point-to-point distance
        if (len(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]]) == len(Y_pred_k_dn[traj_idx])):
            haversine = compute_point_to_point_haversine_distances(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], Y_pred_k_dn[traj_idx])
            
            # Hausdorff distance
            hdist = dst_utils_tsgan.hausdorff_distance(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=np.float32), np.array(Y_pred_k_dn[traj_idx], dtype=np.float32), distance="haversine")
            # EDR distance
            edr = dst_utils_tsgan.edit_distance(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=np.float32), Y_pred_k_dn[traj_idx], eps=0.01)
            # DTW distance
            dtw, path = fastdtw(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=float), Y_pred_k_dn[traj_idx], dist=haversine_dist_in_km)
            # Kendall Tau coefficients
            
            kt_coeff, kt_pval = compute_kendall_tau(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=float), Y_pred_k_dn[traj_idx])
            
            # Mutual Information Score

            mi_score = mutual_information_gps(np.array(Y_test_dn[traj_idx][0:test_seq_len[traj_idx]], dtype=float), Y_pred_k_dn[traj_idx], n_lat_bins=grid_num, n_lon_bins=grid_num)
            #

            mean_dst = np.mean(haversine)
            haversine_dists.append(mean_dst)
            haus_dists.append(hdist)
            edr_dists.append(edr)
            dtw_dists.append(dtw)
            kendalltau_coeffs.append(kt_coeff)
            # js_dists.append(js_dist)
            # kl_divs.append(kl_div)
            mi_scores.append(mi_score)
            
            if (mean_dst >= mmin and mean_dst <=mmax):
                n_mu_n = n_mu_n + 1
            n_traj = n_traj + 1
        else:
            print("Trajectory lengths do not match for trajectory " + str(traj_idx) + ". Skipping...")
            continue
        

    print("Targeted mean: ")
    print("Mean min: " + str(mmin) + " / Mean max: " + str(mmax))
    print("Mean Haversine distance:")
    print(np.mean(haversine_dists))
    print("Mean DTW:")
    print(np.mean(dtw_dists))
    print("Mean Hausdorff:")
    print(np.mean(haus_dists))
    print("Mean EDR:")
    print(np.mean(edr_dists))
    print("Mean Jensen-Shannon Distance (JDS):")
    print(np.mean(js_dists))
    print("Mean KL Divergence:")
    print(np.mean(kl_divs))
    print("---")
    print("Mean Density Error:")
    print(density_error)
    print("Mean Relative Query Error:")
    print(query_error)
    print("Mean Hotspot Query Error:")
    print(hotspot_qerr)
    print("Mean Kendall Tau:")
    print(np.mean(kendalltau_coeffs))
    # print("Mean Coverage Kendall Tau:")
    # print(kendall_tau)
    print("Mean Trip Error:")
    print(trip_error)
    print("Mean Length Error:")
    print(length_error)
    print("Mean Diameter Error:")
    print(diameter_error)
    print("Mean Pattern F1 Error:")
    print(pattern_f1_error)
    print("Mean Pattern Support Error:")
    print(pattern_support_error)
    print("-------")
    print("Mutual Information Score:")
    print(np.mean(mi_scores))
    print("---")
    print("Number of trajectories with MDE in range: " + str(n_mu_n) + " / " + str(n_traj))
    print("Accuracy/Percentage of trajectories with MDE in range: " + str(n_mu_n / n_traj * 100) + "%")
    print("Standard deviation of Haversine distances: " + str(statistics.stdev(haversine_dists,xbar=(mmax+mmin)/2)))
    print("##################################")
    
    
    

Selected dataset: PORTO
grid_num: 20
Targeted mean: 
Mean min: 80 / Mean max: 155
Mean Haversine distance:
125.9867273274228
Mean DTW:
4.25921089685239
Mean Hausdorff:
0.2784359745766815
Mean EDR:
0.0
Mean Jensen-Shannon Distance (JDS):
nan
Mean KL Divergence:
nan
---
Mean Density Error:
0.141094674412876
Mean Relative Query Error:
0.08607233627932148
Mean Hotspot Query Error:
0.8786144122119879
Mean Kendall Tau:
0.59293631931138
Mean Trip Error:
0.21368109576587047
Mean Length Error:
0.38663454193271096
Mean Diameter Error:
0.40051394386468514
Mean Pattern F1 Error:
0.27
Mean Pattern Support Error:
0.685
-------
Mutual Information Score:
2.8915529233692143
---
Number of trajectories with MDE in range: 10 / 10
Accuracy/Percentage of trajectories with MDE in range: 100.0%
Standard deviation of Haversine distances: 12.275904447842167
##################################
Targeted mean: 
Mean min: 170 / Mean max: 350
Mean Haversine distance:
244.0869101543226
Mean DTW:
8.348498061521758
Mean